In [1]:
%pip install openai PyPDF2 fpdf ipywidgets


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%pip install -q gradio openai pypdf2 tiktoken langchain

Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: ffmpy is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
%pip install langchain_community

  Using cached langchain_community-0.2.6-py3-none-any.whl (2.2 MB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
  Using cached marshmallow-3.21.3-py3-none-any.whl (49 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
%pip install PyPDF2 transformers torch



  Using cached torch-2.3.1-cp311-cp311-win_amd64.whl (159.8 MB)
  Using cached sympy-1.12.1-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.3-py3-none-any.whl (1.7 MB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl (228.5 MB)
  Using cached intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl (3.5 MB)
  Using cached tbb-2021.13.0-py3-none-win_amd64.whl (286 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)



[notice] A new release of pip available: 22.3.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
%pip install pypdf PyMuPDF

     -------------------------------------- 290.4/290.4 kB 6.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
def extract_text_from_pdf(pdf_file_path):
    doc = fitz.open(pdf_file_path)
    text = ""
    
    for page in doc:
        page_text = page.get_text()
        text += page_text
        
        # If no text was extracted, use OCR on the page's image
        if not page_text.strip():
            pix = page.get_pixmap()
            img = Image.open(io.BytesIO(pix.tobytes()))
            ocr_text = pytesseract.image_to_string(img)
            text += ocr_text
    
    print(f"Extracted text length from {pdf_file_path}: {len(text)}")
    return text

def save_text_from_pdfs(pdf_folder_path):
    pdf_files = [f for f in os.listdir(pdf_folder_path) if f.lower().endswith('.pdf')]
    for pdf_file in pdf_files:
        pdf_file_path = os.path.join(pdf_folder_path, pdf_file)
        text = extract_text_from_pdf(pdf_file_path)
        if text:
            output_file_path = os.path.join(TEXT_OUTPUT_FOLDER, os.path.basename(pdf_file_path).replace(".pdf", ".txt"))
            with open(output_file_path, 'w', encoding='utf-8') as file:
                file.write(text)
            print(f"Saved text to {output_file_path}")

save_text_from_pdfs(PDF_FOLDER_PATH)

In [ ]:


import os
from langchain.chains.mapreduce import MapReduceDocumentsChain
from langchain.chains import ReduceDocumentsChain, LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.llms import Ollama
from langchain.document_loaders import PyPDFLoader
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from multiprocessing import Pool

# Initialize the language model
llm = Ollama(model="llama3")

# Paths and constants
PDF_FILE_PATH = "SCSL-03-01-T-1283 PART 1.pdf"
FINAL_SUMMARY_FILE = "final_summary.txt"
CHUNK_SIZE = 4000
CHUNK_OVERLAP = 200

# Function to extract and split content from PDF
def extract_and_split_content(pdf_file_path):
    loader = PyPDFLoader(pdf_file_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    return text_splitter.split_documents(documents)

# Prompts for the map and reduce stages
map_template = """chunks prompt here

{content}

CONCISE SUMMARY:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(prompt=map_prompt, llm=llm)

reduce_template = """The following is a set of summaries of legal documents:

{doc_summaries}

final promt here
FINAL SUMMARY:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(prompt=reduce_prompt, llm=llm)

# Setup the map-reduce chain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain=combine_documents_chain,
)

map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    document_variable_name="content",
    reduce_documents_chain=reduce_documents_chain,
    return_intermediate_steps=False,
)

# Function to process a single chunk
def process_chunk(chunk):
    return map_chain.run(content=chunk.page_content)

# Function to summarize the PDF using map-reduce with parallel processing
def summarize_pdf_map_reduce(pdf_file_path):
    # Extract and split the content
    documents = extract_and_split_content(pdf_file_path)
   
    # Use multiprocessing for the map step
    with Pool() as pool:
        chunk_summaries = pool.map(process_chunk, documents)
   
    # Combine the summaries
    combined_summary = [Document(page_content=summary) for summary in chunk_summaries]
   
    # Final reduce step
    final_summary = reduce_documents_chain.run(combined_summary)
   
    # Write the final summary to a file
    with open(FINAL_SUMMARY_FILE, 'w', encoding='utf-8') as file:
        file.write(final_summary)
   
    return final_summary

# Run the summarization
final_summary = summarize_pdf_map_reduce(PDF_FILE_PATH)
print(final_summary)

In [ ]:
import fitz  # PyMuPDF
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# Initialize language model
llm = Ollama(model="llama3")

# Paths and constants
PDF_FILE_PATH = "pdf"
FINAL_SUMMARY_FILE = "final_summary.txt"
CHUNK_SIZE = 4000
CHUNK_OVERLAP = 200
MAX_WORKERS = 4  

def extract_text_from_pdf(pdf_file_path):
    doc = fitz.open(pdf_file_path)
    text = " ".join(page.get_text() for page in doc)
    print(f"Extracted text length: {len(text)}")
    return text

def chunk_text(text, chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_text(text)
    print(f"Number of chunks created: {len(chunks)}")
    return chunks

def summarize_chunk(chunk):
    summarize_template = """Summarize the following text concisely,
    {content}

    SUMMARY:"""
    summarize_prompt = PromptTemplate.from_template(summarize_template)
    summarize_chain = LLMChain(prompt=summarize_prompt, llm=llm)
    return summarize_chain.run(content=chunk)

def summarize_chunks_parallel(chunks):
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        future_to_chunk = {executor.submit(summarize_chunk, chunk): chunk for chunk in chunks}
        summaries = []
        for future in as_completed(future_to_chunk):
            summaries.append(future.result())
    return summaries

def final_summarization(combined_summary):
    reduce_template = """

    {content}

    FINAL SUMMARY:"""
    reduce_prompt = PromptTemplate.from_template(reduce_template)
    reduce_chain = LLMChain(prompt=reduce_prompt, llm=llm)
    return reduce_chain.run(content=combined_summary)

def summarize_pdf(pdf_file_path):
    start_time = time.time()

    text = extract_text_from_pdf(pdf_file_path)
    if not text:
        print("No text extracted from PDF")
        return

    text_chunks = chunk_text(text)
    if not text_chunks:
        print("No chunks formed")
        return

    print("Summarizing chunks...")
    chunk_summaries = summarize_chunks_parallel(text_chunks)
    
    print("Generating final summary...")
    combined_summary = " ".join(chunk_summaries)
    final_summary = final_summarization(combined_summary)
    
    with open(FINAL_SUMMARY_FILE, 'w', encoding='utf-8') as file:
        file.write(final_summary)
    
    end_time = time.time()
    print(f"Total processing time: {end_time - start_time:.2f} seconds")
    
    return final_summary

if __name__ == "__main__":
    try:
        final_summary = summarize_pdf(PDF_FILE_PATH)
        if final_summary:
            print("Final Summary:")
            print(final_summary)
        else:
            print("Summarization process failed.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [ ]:
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
import time

# Initialize the language model
llm = Ollama(model="llama3")  # Assuming Ollama is used here

# Paths and constants
TEXT_FOLDER_PATH = "RUF/text_files"  # Folder containing text files
SUMMARY_OUTPUT_FOLDER = os.path.join(TEXT_FOLDER_PATH, "../summaries")
os.makedirs(SUMMARY_OUTPUT_FOLDER, exist_ok=True)
CHUNK_SIZE = 4000
CHUNK_OVERLAP = 200
MAX_WORKERS = 8  # Number of parallel workers


In [ ]:
def chunk_text(text, chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_text(text)
    print(f"Number of chunks created: {len(chunks)}")
    return chunks

def summarize_chunk(chunk):
    summarize_template = """
    {content}

    SUMMARY:"""
    summarize_prompt = PromptTemplate.from_template(summarize_template)
    summarize_chain = LLMChain(prompt=summarize_prompt, llm=llm)
    return summarize_chain.run(content=chunk)

def reduce_summaries(chunk_summaries):
    combined_summary = " ".join(chunk_summaries)

    reduce_template = """

    {content}

    FINAL SUMMARY:"""
    reduce_prompt = PromptTemplate.from_template(reduce_template)
    reduce_chain = LLMChain(prompt=reduce_prompt, llm=llm)
    return reduce_chain.run(content=combined_summary)

def summarize_text_file(text_file_path, output_folder):
    with open(text_file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    if not text:
        print(f"No text extracted from {text_file_path}")
        return None

    text_chunks = chunk_text(text)
    if not text_chunks:
        print(f"No chunks formed for {text_file_path}")
        return None

    print(f"Summarizing chunks for {text_file_path}...")
    chunk_summaries = [summarize_chunk(chunk) for chunk in text_chunks]

    print(f"Generating final summary for {text_file_path}...")
    final_summary = reduce_summaries(chunk_summaries)

    output_file_path = os.path.join(output_folder, os.path.basename(text_file_path).replace(".txt", "_summary.txt"))
    with open(output_file_path, 'w', encoding='utf-8') as file:
        file.write(final_summary)

    print(f"Processed {text_file_path}")

    return final_summary

def summarize_all_texts(text_folder_path):
    text_files = [f for f in os.listdir(text_folder_path) if f.lower().endswith('.txt')]

    all_summaries = []

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        future_to_text_file = {executor.submit(summarize_text_file, os.path.join(text_folder_path, text_file), SUMMARY_OUTPUT_FOLDER): text_file for text_file in text_files}

        for future in as_completed(future_to_text_file):
            text_file = future_to_text_file[future]
            try:
                summary = future.result()
                if summary:
                    all_summaries.append(summary)
            except Exception as e:
                print(f"An error occurred while processing {text_file}: {str(e)}")

    # Generate a final summary of all individual summaries
    if all_summaries:
        for i, summary in enumerate(all_summaries):
            combined_output_path = os.path.join(SUMMARY_OUTPUT_FOLDER, f"final_summary_{i+1}.txt")
            with open(combined_output_path, 'w', encoding='utf-8') as file:
                file.write(summary)
            print(f"Final Summary for {combined_output_path}:")
            print(summary)

summarize_all_texts(TEXT_FOLDER_PATH)


In [ ]:
import os
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter

llm = Ollama(model="llama3")

SUMMARY_FOLDER_PATH = "summaries"
CHUNK_SIZE = 500
CHUNK_OVERLAP = 200

def read_summary_files(folder_path):
    summaries = []
    for filename in os.listdir(folder_path):
        if filename.endswith("_summary.txt"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                summaries.append(file.read())
    return summaries

def combine_summaries(summaries):
    combined_text = "\n\n".join(summaries)
    return combined_text

def chunk_text(text, chunk_size, chunk_overlap):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_text(text)
    return chunks

def summarize_chunks(chunks):
    chunk_summaries = []
    summary_template = """Summarize the following text, highlighting the key points:

    {text}

    Summary:"""

    for chunk in chunks:
        prompt = PromptTemplate.from_template(summary_template)
        chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
        summary = chain.run(text=chunk)
        chunk_summaries.append(summary)

    return chunk_summaries

def combine_chunk_summaries(chunk_summaries):
    combined_template = """

    {content}

    FINAL COMPREHENSIVE SUMMARY:"""

    combined_prompt = PromptTemplate.from_template(combined_template)
    combined_chain = LLMChain(prompt=combined_prompt, llm=llm, verbose=True)
    return combined_chain.run(content="\n\n".join(chunk_summaries))

# Read all summary files
all_summaries = read_summary_files(SUMMARY_FOLDER_PATH)

# Combine all summaries into one text
combined_text = combine_summaries(all_summaries)

# Chunk the combined text
chunks = chunk_text(combined_text, CHUNK_SIZE, CHUNK_OVERLAP)

# Summarize each chunk
chunk_summaries = summarize_chunks(chunks)

# Combine chunk summaries into a final summary
final_summary = combine_chunk_summaries(chunk_summaries)

# Print the final summary
print(final_summary)

# Save the final summary to a file
output_path = os.path.join(SUMMARY_FOLDER_PATH, "final_combined_summary.txt")
with open(output_path, 'w', encoding='utf-8') as file:
    file.write(final_summary)

print(f"Final combined summary saved to: {output_path}")
